### Projeto de Escalonamento ###

Neste projeto, será simulado o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, tem-se uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Perceba que é instanciado o escalonador. Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

### Projeto: ###


1 - Calcular o tempo de execucao medio e tempo de resposta 

2 - Implementar o escalonador por FIFO e SJF e verificar seus tempos de execução e espera.

3 - Fazer em outra célula uma implementação do STCF e Round Robin, alterando o p.roda() para receber o quantum

4 - Implementar Round Robin, Stride, Lottery, MLFQ


In [2]:
import numpy as np
import random
from operator import attrgetter
#PARTE DO FIFO E SJF, MUDEI AS TIMESLICES DOS PROCESSOS PRA TESTAR
#Autor: Kennedy Coelho Nolasco
class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria aora 
        self.chegada = 0

    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    

class FIFO(object): # escalonador FIFO
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        
        a =1  # Não usei esse método por enquanto pois todos os processos ja chegam prontos e não saem desse estado 
        
        
    def proximo(self):
       
        self.prontos = [i for i in self.prontos if i.tam>0] #eu removo todos os processos que tem timeslice igual ou menor que 0
        p = min(self.prontos,key=attrgetter('chegada')) #chamo o processo que chegou primeiro, nesse caso nao importa muito pois todos os processos chegaram no tempo 0
        return p #processo p eh escolhido
    
class SJF(object): # escalonador SJF
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        
        a =1  # Não usei esse método por enquanto pois todos os processos ja chegam prontos e não saem desse estado 
        
        
    def proximo(self):
        
        self.prontos = [i for i in self.prontos if i.tam>0] #eu removo todos os processos que tem timeslice igual ou menor que 0
        p = min(self.prontos,key=attrgetter('tam')) #chamo o processo que tem menor tempo para terminar
        return p #processo p eh escolhido

nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([4,2,3,1])


total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,0)) #cria uma lista procs de Processos

quantum = 2
tempoBloq = 1

escalonador = FIFO(procs) #troque escalonador pelo seu escalonador
bloqueados = []
tempo = 0 
tempoM = 0 #tempo medio de execucao 
tempoR = 0 #tempo de resposta de cada processo
random.seed(0)
x = len(procs)
while total>0:
    p = escalonador.proximo()
    if(p is not None):
        tempoR = abs(tempo - p.chegada)# tempo de resposta é o tempo antes de ser executado
        print('Tempo de resposta de',p.nome ,':', tempoR) #tempo de resposta é o tempo antes dele ser executado pela primeira vez
        rodou, _ = p.roda() #nao tem preempcao logo nao tem quantum
        if(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        tempo+=rodou 
        tempoM += abs(tempo - p.chegada) #tempo de execução é  o tempo depois de ser executado
        
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
print('Tempo médio de execução',tempoM/x)
    

Tempo de resposta de D : 0
D  rodou por  1  timeslice, faltam  0
Tempo de resposta de B : 1
B  rodou por  2  timeslice, faltam  0
Tempo de resposta de C : 3
C  rodou por  3  timeslice, faltam  0
Tempo de resposta de A : 6
A  rodou por  4  timeslice, faltam  0
Tempo médio de execução 5.0


In [8]:
import numpy as np
import random
import copy
from operator import attrgetter
#PARTE DO STCF E ROUND ROBIN, MUDEI AS TIMESLICES DOS PROCESSOS PRA TESTAR
class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria aora 
        self.chegada = 0

    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    

class STCF(object): #escalonador STCF, bem parecido com o SJF, única diferença é que agora é usado o quantum
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        
        a =1  # Não usei esse método por enquanto pois todos os processos ja chegam prontos e não saem desse estado 
        
    def remove(self): #funcao que remove todos os processos terminados caso precise
        self.prontos = [i for i in self.prontos if i.tam>0] #eu removo todos os processos que tem timeslice igual ou menor que 0    
    def proximo(self):
        
        p = min(self.prontos,key=attrgetter('tam')) #chamo o processo que tem menor tempo para terminar
        return p #processo p eh escolhido

class ROUNDROBIN(object): # escalonador round robin
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        
        a =1 # Não usei esse método por enquanto pois todos os processos ja chegam prontos e não saem desse estado 
        
    def remove(self): #funcao que remove todos os processos terminados caso precise
        self.prontos = [i for i in self.prontos if i.tam>0] #eu removo todos os processos que tem timeslice igual ou menor que 0

    def proximo(self,atual): #recebe o indice do processo seguinte a ser executado
        
        p = self.prontos[atual] #chamo o processo a seguir do round robin
        return p #processo p eh escolhido


nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([4,2,3,1])


total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,0)) #cria uma lista procs de Processos

quantum = 2
tempoBloq = 1

escalonador = STCF(procs) #troque escalonador pelo seu escalonador
bloqueados = []
tempo = 0 
tempoM = 0 #tempo medio de execucao
tempoR = 0 #tempo de resposta
random.seed(0)
a = 0
b = 0 #contador
x = len(procs) #numero de processos
tempoI =  copy.deepcopy(procs) # copio os processos originais com os tempos iniciais
classe = isinstance(escalonador,STCF) #verifico se vou rodar o bloco de codigo do round robin ou do STCF
if(classe):
    while total>0:
        p = escalonador.proximo()
        if(p is not None):
            prox = next(x for x in tempoI if x.nome == p.nome ) #chamo o cara que tem o nome do processo atual na lista de processos originais
            if(p.tam == prox.tam): #verifico o tempo original dele pra ver se ele ja foi executado
                tempoR = abs(tempo-p.chegada)# tempo de resposta é o tempo antes de ser executado
                print('Tempo de resposta de',p.nome ,':', tempoR) #
            rodou, _ = p.roda(quantum) #adicione quantum como parâmetro, por enquanto nao temos E/S
            if(p.tam>0):
                escalonador.pronto(p)
            if(p.tam==0):
                escalonador.remove() #se o cara atual ja terminou de ser executado eu removo ele
            total-=rodou
            tempo+=rodou
            if p.tam==0: #eu só somo o tempo  de execucao agora quando o processo sai da minha lista de processos
                tempoM += abs(tempo - p.chegada) #tempo de execução é  o tempo depois de ser executado
        else:
            #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
            tempo+=1
        a = p
else:
    while total>0:
        y = len(escalonador.prontos)
        p = escalonador.proximo(b)

        if(p is not None):
            prox = next(x for x in tempoI if x.nome == p.nome ) #mesma coisa que no STCF
            if(p.tam == prox.tam): # verifico o tempo original dele pra ver se foi executado antes
                tempoR = abs(tempo-p.chegada)# tempo de resposta é o tempo antes de ser executado
                print('Tempo de resposta de',p.nome ,':', tempoR) #
            rodou, _ = p.roda(quantum) #adicione quantum como parâmetro, por enquanto nao temos E/S

            if(p.tam>0):
                escalonador.pronto(p)
            if(p.tam==0):
                escalonador.remove() #se o cara atual ja terminou de ser executado eu removo ele
            total-=rodou
            tempo+=rodou
            if p.tam==0: #eu só somo o tempo  de execucao agora quando o processo sai da minha lista de processos
                tempoM += abs(tempo - p.chegada) #tempo de execução é  o tempo depois de ser executado
        else:
            #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
            tempo+=1


        if y > len(escalonador.prontos):
            b-=1
        b+=1
        if b >=len(escalonador.prontos):
            b=0
        a = p
print('Tempo médio de execução',tempoM/x)


    

Tempo de resposta de D : 0
D  rodou por  1  timeslice, faltam  0
Tempo de resposta de B : 1
B  rodou por  2  timeslice, faltam  0
Tempo de resposta de C : 3
C  rodou por  2  timeslice, faltam  1
C  rodou por  1  timeslice, faltam  0
Tempo de resposta de A : 6
A  rodou por  2  timeslice, faltam  2
A  rodou por  2  timeslice, faltam  0
Tempo médio de execução 5.0
